In [ ]:
!pip install findspark

In [3]:
!pip install pyspark==3.5.4

In [4]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
import socket
from pyspark import SparkConf, SparkContext

# Проверка доступности портов


def check_port(port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('127.0.0.1', port))
    sock.close()
    return result == 0


print("Порт 7077 доступен:", check_port(7077))
print("Порт 7078 доступен:", check_port(7078))

# Запуск с диагностикой
conf = SparkConf().setAppName("DiagApp").setMaster("local[1]")
sc = SparkContext(conf=conf)

print("SparkContext создан успешно")
print("Версия Spark:", sc.version)

rdd = sc.parallelize([1, 2, 3, 4, 5])
print("RDD создан, количество партиций:", rdd.getNumPartitions())

result = rdd.mean()
print("Результат:", result)

sc.stop()

In [ ]:
from pyspark.sql import SparkSession
import findspark
findspark.init()


# Самая простая конфигурация
spark = SparkSession.builder \
    .appName("SimpleTest") \
    .master("local") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

sc = spark.sparkContext

# Быстрый тест
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)
print("Количество элементов:", rdd.count())
print("Среднее значение:", rdd.mean())

spark.stop()

In [5]:
import socket
from pyspark import SparkConf, SparkContext
import random

# Функция для поиска свободного порта


def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]


# Используем случайные свободные порты
driver_port = find_free_port()
blockmanager_port = find_free_port()

print(
    f"Используем порты: driver={driver_port}, blockmanager={blockmanager_port}")

conf = SparkConf() \
    .setAppName("FixedApp") \
    .setMaster("local[1]") \
    .set("spark.driver.host", "localhost") \
    .set("spark.driver.bindAddress", "127.0.0.1") \
    .set("spark.driver.port", str(driver_port)) \
    .set("spark.blockManager.port", str(blockmanager_port)) \
    .set("spark.network.timeout", "300s") \
    .set("spark.executor.heartbeatInterval", "30s") \
    .set("spark.sql.adaptive.enabled", "true") \
    .set("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

sc = SparkContext(conf=conf)

# Тестируем
try:
    rdd = sc.parallelize([1, 2, 3, 4, 5])
    print("RDD создан успешно")
    mean_value = rdd.mean()
    print("Среднее значение элементов в RDD:", mean_value)
except Exception as e:
    print("Ошибка:", e)
finally:
    sc.stop()

Используем порты: driver=49584, blockmanager=49585
RDD создан успешно
Среднее значение элементов в RDD: 3.0


In [6]:
from pyspark import SparkContext

# Самый простой возможный вариант
sc = SparkContext("local", "test")

# Вместо mean() вычисляем вручную
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)

# Вычисляем сумму и count отдельно
total = rdd.sum()
count = rdd.count()

if count > 0:
    mean_manual = total / count
    print(f"Среднее значение (вычислено вручную): {mean_manual}")
else:
    print("RDD пуст")

sc.stop()

Среднее значение (вычислено вручную): 3.0


In [7]:
sc.stop()